# Travaux pratiques - Classification des sols

**L’objectif** de cette deuxième séance de travaux pratiques est de vous faire manipuler des données réelles pour une tâche de classification de couverture des sols à partir d'images aériennes ou satellitaires.

Références externes utiles :
- [Documentation NumPy](https://docs.scipy.org/doc/numpy/user/index.html)  
- [Documentation SciPy](https://docs.scipy.org/doc/scipy/reference/)  
- [Documentation MatPlotLib](http://matplotlib.org/)  
- [Documentation de scikit-learn](http://scikit-learn.org/stable/index.html)  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn

## Évaluation des modèles et corrélations spatiales

Pour commencer, nous allons considérer le jeu de données *Pavia University*. Il s'agit d'une image hyperspectrale annotée de $610\times340$ pixels, comprenant 103 bandes spectrales. L'acquisitition a été réalisée à l'aide du spectromètre ROSIS-3, et couvre une partie de l'université de Pavie en Italie. Le jeu de données total comporte ainsi 42776 pixels annotés dans 9 classes différentes : *asphalt*, *meadows*, *gravel*, *trees*, *metal sheet*, *bare soil*, *bitumen*, *brick*, et *shadow*.

In [ ]:
from scipy.io import loadmat
from urllib.request import urlretrieve
urlretrieve("https://www.ehu.eus/ccwintco/uploads/e/ee/PaviaU.mat", "PaviaU.mat")
urlretrieve("https://www.ehu.eus/ccwintco/uploads/5/50/PaviaU_gt.mat", "PaviaU_gt.mat")

In [ ]:
pavia_image = loadmat("PaviaU.mat")["paviaU"]
pavia_gt = loadmat("PaviaU_gt.mat")["paviaU_gt"]

In [ ]:
rgb_bands = (55,41,12)
pavia_rgb = pavia_image[:,:,rgb_bands]

fig = plt.figure(figsize=(12, 12))
fig.add_subplot(1,2,1)
plt.imshow((pavia_rgb - pavia_rgb.min()) / pavia_rgb.max())
plt.title("Image pseudo-RGB")
fig.add_subplot(1,2,2)
plt.imshow(pavia_gt, cmap="Pastel1")
plt.title("Vérité terrain")
plt.show()

Nous pouvons maintenant mettre en forme le jeu de données pour son traitement avec scikit-learn. L'opération dans notre cas consiste à formater les pixels (de train puis de test) sous forme d'une liste de vecteurs (chaque vecteur correspondant aux réflectances de 103 bandes spectrales). C'est donc un simple redimensionnement avec numpy. Attention toutefois, une subtilité de ce jeu de données est que la classe 0 correspond en réalité aux pixels *non annotés* : il faut donc les retirer (on ne souhaite ni s'entraîner dessus, ni s'évaluer dessus).

In [ ]:
# Le jeu de données contient seulement les pixels pour lesquels la vérité terrain est non-nulle 
X = pavia_image[pavia_gt != 0].reshape(-1, 103)
y = pavia_gt[pavia_gt != 0].reshape(-1)

nonzero_coordinates = np.nonzero(pavia_gt)
coordinates = np.array(list(zip(*nonzero_coordinates)))

Pour commencer, tirons aléatoirement 10% des pixels dans l'image pour constituer un jeu d'apprentissage. Le reste des pixels constituera le jeu de test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, X_coords_train, X_coords_test, y_train, y_test = train_test_split(X, coordinates, y, train_size=0.1)

### Question

Que représente la matrice `coordinates` ?

### Correction

Il s'agit de la matrice qui a pour valeur le tuplet $(i,j)$ aux coordonnées $(i,j)$. Autrement dit, chaque élément représente ses propres coordonnées.

### Question

Appliquer une classification par k-plus proches voisins avec $k=5$ en utilisant comme *features* d'entrée le spectre représenté par chaque pixel. La documentation du module *[Neighbors](https://scikit-learn.org/stable/modules/neighbors.html)* de scikit-learn peut être utile.

### Correction

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

### Question

Appliquer une classification par k-plus proche voisin avec $k=5$ en utilisant comme *features* d'entrée les **coordonnées** des pixels (et pas les valeurs de réflectance).

### Correction

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_coords_train, y_train)
clf.score(X_coords_test, y_test)

### Question

Que peut-on en déduire sur les corrélations spatiales dans le jeu de données ? Comment faut-il diviser le jeu de données en train/test pour que l'évaluation ne soit pas biaisée ?

### Correction

Il séparer le jeu de données en deux régions disjointes, sans quoi le test se fera sur des pixels physiquement très proches des pixels utilisées pour l'entraînement, ce qui est très optimiste ! La preuve, en comparant un pixel avec ses cinq voisins les plus proches dans le jeu d'apprentissage, on obtient >90% de bonne classification alors que le modèle ne prend même pas en compte le spectre de ce qui se trouve au sol.

Pour que notre évaluation soit plus rigoureuse, nous allons donc séparer le jeu de données, de façon plus ou moins arbitraire, en créant une grille rectangulaire de 10 cases. Chaque bloc de la grille sera soit en train, soit en test :

In [ ]:
groups_2d = np.zeros_like(pavia_gt)
h, w = groups_2d.shape
idx = 0
for i in range(0, h, h // 5):
    for j in range(0, w, int(w / 2)):
        groups_2d[i:i+h//5, j:j+int(w / 2)] = idx
        idx += 1

groups = groups_2d[pavia_gt != 0]

fig = plt.figure(figsize=(15, 9))
plt.imshow((pavia_rgb - pavia_rgb.min()) / pavia_rgb.max())
plt.imshow(groups_2d, cmap="tab10", alpha=0.5)
plt.title("Groupes")
plt.show()

### Question

Quelle est la stratégie de validation croisée la plus adaptée ? Vous pouvez consulter la liste des stratégies implémentées par [sklearn](https://scikit-learn.org/stable/api/sklearn.model_selection.html) pour vous aider.

### Correction

Dans ce cas précis, le $k$-fold par groupe (*GroupKFold*) est la méthode la plus appropriée.

### Question

Divisez le jeu de données en deux (apprentissage et test) à l'aide de l'algorithme de validation croisé que vous avez choisi. Vous pouvez utiliser la documentation de [scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators-for-grouped-data) pour vous aider.

Appliquez ensuite le classifieur de votre choix de scikit-learn en l'entraînant sur le jeu de données d'apprentissage, puis évaluez le jeu de données de validation. N'oubliez pas de faire attention à la métrique à utiliser (le jeu de données est-il équilibré ? y a-t-il des classes plus difficiles à prédire que d'autres ?).

Répétez cette opération pour toutes les *folds* de la validation croisée. Qu'observez-vous ?

### Correction

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GroupKFold
from sklearn.metrics import classification_report

gkf = GroupKFold(n_splits=3)
scores = []

for idx, (train, test) in enumerate(gkf.split(X, y, groups=groups)):
    print(f"=== Fold {idx+1} ===")
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]

    clf = LinearSVC()
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(score)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred, zero_division=1.0, labels=np.unique(y), target_names=["Asphalt", "Meadows", "Gravel", "Trees", "Painted metal sheets", "Bare Soil" , "Bitumen", "Self-Blocking Bricks", "Shadows"]))
    scores.append(score)

    # Identifiant des blocs sur lesquels tester
    group_idx = [i for i in np.unique(groups[test])]
    
    predictions = np.zeros_like(pavia_gt)
    group_mask = np.bitwise_and(pavia_gt != 0, np.isin(groups_2d, group_idx))
    X_eval = pavia_image[group_mask]
    predictions[group_mask] = clf.predict(X_eval)
    
    fig = plt.figure(figsize=(10, 10))
    fig.add_subplot(1,2,1)
    plt.imshow(pavia_gt, cmap="Pastel1")
    plt.title("Vérité terrain")
    fig.add_subplot(1,2,2)
    plt.imshow(predictions, cmap="Pastel1")
    plt.title("Prédictions")
    plt.show()

### Correction

Les performances varient beaucoup pour chaque *fold* car les classes n'y sont pas uniformément représentées.

### Question

Tracez la courbe du score moyen du modèle estimé par la validation croisée avec $k$ partitions en augmentant $k$, par exemple de 1 à 10. Représentez également l'écart-type de ce score. Que constatez-vous ?

**Note**: si les temps de calcul sont trop longs avec une SVM linéaire (`LinearSVC`), vous pouvez la remplacer par une approximation utilisant la descente de gradient : `sklearn.linear_model.SGDClassifier` (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html). Vous pouvez aussi utiliser la fonction `cross_val_score()` de scikit-learn en spécifiant `n_jobs>1` pour paralléliser les calculs sur plusieurs cœurs.

### Correction

En augmentant le nombre de partitions dans la validation croisée, l'estimation du score se stabilise et devient plus robuste : la variance augmente mais l'estimateur converge vers le « vrai » score de généralisation.

In [ ]:
from tqdm.auto import tqdm
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

n_splits = range(2, 11)
mean_score = []
std_score = []

for k in tqdm(n_splits):
    gkf = GroupKFold(n_splits=k)
    clf = SGDClassifier(alpha=1e-2)
    scores = cross_val_score(clf, X, y, cv=gkf.split(X, y, groups=groups), n_jobs=-1)
    mean_score.append(np.mean(scores))
    std_score.append(np.std(scores))

mean_score, std_score = np.array(mean_score), np.array(std_score)

plt.fill_between(n_splits, mean_score - std_score, mean_score + std_score, alpha=0.2)
plt.plot(n_splits, mean_score)
plt.xlabel("$k$")
plt.ylabel("Accuracy")
plt.show()

## Recherche d'hyperparamètres

La SVM linéaire a pour principal hyperparamètre le facteur de régularisation $C$. Pour rappel, contrôle le compromis entre la largeur de la marge et le nombre d'erreurs de classification commises par le modèle.

Pour déterminer la valeur de cet hyperparamètre, nous allons effectuer une recherche d'hyperparamètres par grille (*GridSearch*). scikit-learn implémente un algorithme de recherche systématique qui gère automatiquement la validation croisée : [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Pour commencer, nous allons définir la liste des valeurs que nous voulons tester pour $C$. Par exemple, nous pouvons choisir 5 valeurs différentes, de $10^{-3}$ à 10.

In [ ]:
grid = {'C': [10e-3, 10e-2, 0.1, 1.0, 10]}

Nous pouvons maintenant définir la recherche par grille. Attention, on ne peut pas se contenter de la validation croisée par défaut de scikit-learn (qui effectue un $k$-fold). On va explicitement spécifiquer qu'il faut utiliser la validation croisée groupée définie plus haut, à l'aide du paramètre `cv=` :

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

gkf = GroupKFold(n_splits=5)
grid_search = GridSearchCV(LinearSVC(), grid, cv=gkf.split(X, y, groups=groups), n_jobs=-1)

### Question

Combien de modèles vont être entraînés lors de cette recherche par grille ?

### Correction

En supposant un $k$-fold avec $k=5$, il y aura 26 modèles entraînés ($k$ modèles pour chaque point de la grille, soit $5\times5$, plus un modèle final ré-entraîné avec le meilleur hyperparamètre).

### Question

Appliquer cette *GridSearch* sur le jeu de données. Quelle est la valeur optimale de $C$ ?

### Correction

In [ ]:
grid_search.fit(X, y)

In [ ]:
grid_search.best_params_

### Clustering

Dans ce cas, le jeu de données est annoté, c'est-à-dire que l'on dispose d'étiquettes de classe pour chaque observation. Cela nous permet d'entraîner des modèles *supervisés*. Toutefois, ce n'est pas toujours le cas. Si nous n'avions pas d'étiquettes, une option consiste à réaliser une classification automatique à l'aide d'une méthode de *clustering*, par exemple $k$-*means*. Dans ce cas, il n'y a pas forcément d'intérêt à séparer le jeu de données en apprentissage et validation, car nous n'avons pas de données annotées sur lesquelles calculer une vérité terrain.

Avec scikit-learn, il est possible d'utiliser l'objet [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) pour utiliser cet algorithme :

In [ ]:
from sklearn.cluster import KMeans

X = pavia_image.reshape(-1, 103)

kmeans = KMeans(n_clusters=10)
labels = kmeans.fit_predict(X)

## Question

À l'aide de matplotlib, affichez la carte de l'université de Pavie colorisé selon les clusters déterminés par K-Means. Comparez visuellement à la vérité terrain pour différentes valeurs de $k$. Qu'en pensez-vous ?

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.add_subplot(1,2,1)
plt.imshow((pavia_rgb - pavia_rgb.min()) / pavia_rgb.max())
plt.title("Image pseudo-RGB")
fig.add_subplot(1,2,2)
plt.imshow(labels.reshape(*pavia_gt.shape), cmap="Pastel1")
plt.title("K-Means")
plt.show()

## Pour aller plus loin : classification d'images sur EuroSAT

Pour aller plus loin, nous allons nous intéresser au jeu de
données [EuroSAT](https://github.com/phelber/EuroSAT), qui est une
base d’images contenant :

- 10 classes (bâtiments industriels, bâtiments
  résidentiels, cultures saisonnières, cultures permanentes, rivières,
  lacs & mers, végétation herbacée, routes/autoroutes, pâturages et
  forêts).  
- Il y a 270000 images au total (162000 pour l'apprentissage, 5400 pour la validation et 5400 pour l'évaluation).
- Les images sont de taille $ 64\times64 $. Les images que
  nous allons utiliser sont en couleur (RGB) mais il existe une version
  multispectrale utilisant 13 longueurs d’onde différentes.  

Pour simplifier la prise en mains, nous allons la bibliothèque [`datasets`](https://huggingface.co/docs/datasets/v1.2.0/index.html) qui permet d'importer directement ce jeu de données en une seule commande. Commençons par visualiser quelques images :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from datasets import load_dataset
eurosat = load_dataset("blanchon/EuroSAT_RGB")

fig = plt.figure(figsize=(18, 6))
for idx, data in enumerate(eurosat['train'].shuffle()):
    if idx >= 10:
        break
    fig.add_subplot(1, 10, idx+1)
    plt.imshow(data['image'])
    plt.axis("off")
    plt.title(data['filename'].split('_')[0])
plt.show()

Il s'agit comme indiqué précédemment d'images Sentinel-2 à 10m/px de résolution. Plus précisément, il s'agit des bandes dans le domaine visible, formant une pseudo-image RGB.

La fonction `create_dataset` ci-dessous permet de convertir le jeu de données EuroSAT au format attendu par scikit-learn, c'est-à-dire des tableaux NumPy. Elle transforme le jeu de données en deux tableaux `X` (contenant les images) et `y` (content les étiquettes des classes). Les images transformées en descripteurs (*features*). Pour l'instant, on se contente de redimensionner les images au format $32\times32$ et de la transformer en vecteur. 

In [ ]:
from PIL import Image
# Exemple de redimensionnement d'image avec PIL
im = eurosat['train'][0]['image']
im.resize((32, 32))

In [ ]:
from tqdm.auto import tqdm

def create_dataset(dataset):
    X, y = [], []
    for data in tqdm(dataset):
        image, label = data['image'], data['label']
        features = np.array(image.resize((32, 32))).ravel()
        X.append(features)
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = create_dataset(eurosat['train'])
X_val, y_val = create_dataset(eurosat['validation'])
X_test, y_test = create_dataset(eurosat['test'])

### Question

Utilisez le classifieur de votre choix de scikit-learn et appliquez-le sur le jeu de données. Comparez les performances de différents classifieurs avec différents paramètres *sur le jeu de validation*. N'oubliez pas de faire attention à la métrique à utiliser (le jeu de données est-il équilibré ? y a-t-il des classes plus difficiles à prédire que d'autres ?).

### Correction

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train, y_train)
svm.score(X_val, y_val)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_val, y_val)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_val, y_val)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_val, rf.predict(X_val)))

### Question : pour aller encore plus loin

Répétez l'expérience précédente mais en utilisant d'autres descripteurs pour les images. Par exemple, vous pouvez utiliser les descripteurs HOG de scikit-image (`skimage.feature.hog`), d'autres facteurs de redimensionnement de l'image,  un histogramme de couleurs ou les descripteurs profonds calculés à l'aide de la fonction `extract_deep_features` ci-dessous. Quels descripteurs obtiennent les meilleurs résultats ?

In [ ]:
import torch
import torchvision
from torchvision.models import efficientnet_b0
from torchvision.models import EfficientNet_B0_Weights

net = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
net.classifier = torch.nn.Sequential(*list(net.classifier.children())[:-3])
print(net)

def extract_deep_features(net, image):
    batch = EfficientNet_B0_Weights.DEFAULT.transforms()(image).unsqueeze(0)
    if torch.cuda.is_available():
        net = net.to("cuda")
        batch = batch.to("cuda")
    with torch.no_grad():
        return net(batch).squeeze().to("cpu").numpy()

extract_deep_features(eurosat['train'][0]['image']).shape

### Correction

In [ ]:
def create_dataset(dataset):
    X, y = [], []
    for data in tqdm(dataset):
        image, label = data['image'], data['label']
        features = extract_deep_features(net, image)
        #features = np.array(image.resize((8,8))).ravel()
        #features = hog(image, cells_per_block=(1,1), channel_axis=-1)
        X.append(features)
        y.append(label)
    return np.array(X), np.array(y)

X_train, y_train = create_dataset(eurosat['train'])
X_val, y_val = create_dataset(eurosat['validation'])
X_test, y_test = create_dataset(eurosat['test'])



In [ ]:
from sklearn.linear_model import SGDClassifier

svm = SGDClassifier(alpha=1e-2)
svm.fit(X_train, y_train)
svm.score(X_val, y_val)

from sklearn.metrics import classification_report

print(classification_report(y_val, svm.predict(X_val)))

## Question

Évaluez votre meilleur modèle sur le jeu de test.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_val, y_val)